In [3]:
## Bu hücre bloğu ses dosyasını oluşturmak için kullanılmaktadır. ##

'''
ELM 368 Dönem Projesi - 2020 

Yusuf Sami Açıkgöz - 151024005
Nida Karaçoban - 161024047
Mustafa Akın Akarsu - 161024064

! Önemli Uyarılar !

Birinci hücredeki kodu çalıştırdığınız zaman kodu çalıştırdığınız klasörün içine wavio.write() fonksiyonu içinde isimlendirilen
.wav dosyasını oluşturur. İkinci hücre bloğunu çalıştırmadan önce 'wawfile.read' komutu içinde incelemek istediğiniz 
wav dosyasının konumunun olduğuna emin olunuz !!!
''' 
## Gerekli kütüphanelerin eklenmesi ##
import sounddevice as sd
import numpy as np 
import matplotlib.pyplot as plt
from scipy.fftpack import fft , ifft
import wavio

pi = np.pi
fs = 44100 # oluşturacağımız ses dosyasının örneklenme frekansı

n = np.arange(0,0.4,1/fs) # uzun süreli sesler için aralık belirlenmesi
n2 = np.arange(0,0.25,1/fs) # kısa süreli sesler için aralık belirlenmesi

'''Aşağıdaki 3 frekans değerini ve wavio.write() fonksionu içerisindeki isimlendirmeyi değiştirerek yeni ses dosyaları
oluşturabilir ve nota tahmini kısmında yeni oluşturduğunuz ses dosyaları için denemeler yapabilirsiniz''' 

# kırmızı balık gölde şarkısında kullanılacak notaların frekans değerlerinin girilmesi #
f = 698.456 #FA5 = F5 
f2 = 783.991 # SOL5 = G5
f3 = 880.000 # LA5 = A5

# uzun süreli sesler için ses sinyallerinin oluşturulması #
tam_fa = np.sin(2*pi*f*n)
tam_sol = np.sin(2*pi*f2*n) 
tam_la = np.sin(2*pi*f3*n)

# kısa süreli sesler için ses sinyallerinin oluşturulması #
yarım_fa = np.sin(2*pi*f*n2)
yarım_sol  = np.sin(2*pi*f2*n2)
yarım_la  = np.sin(2*pi*f3*n2)

# nota geçişleri arasında boşluk oluşması için delay dizilerinin tanımlanması #
delay = np.zeros(4410)
delay2 = np.zeros(11025)

# oluşturulan notalara delay değerlerinin eklenmesi #
tam_fa_delay = np.append(tam_fa,delay)
tam_sol_delay = np.append(tam_sol,delay)
tam_la_delay = np.append(tam_la,delay)

yarım_fa_delay = np.append(yarım_fa,delay2)
yarım_sol_delay = np.append(yarım_sol,delay2)
yarım_la_delay = np.append(yarım_la,delay2)

# oluşturulan seslerin 'kırmızı balık gölde' şarkısını oluşturacak şekilde birleştirilmesi #

# KIR-MI-ZI-BA-LIK-GÖL-DE = FA-FA-SOL-FA-SOL-LA-SOL = F5-F5-G5-F5-G5-A5-G5
y = np.append(yarım_fa_delay,yarım_fa_delay)
y2 = np.append(y,yarım_sol_delay)
y3 = np.append(y2,yarım_fa_delay)
y4 = np.append(y3,yarım_sol_delay)
y5 = np.append(y4,tam_la_delay)
y6 = np.append(y5,tam_sol_delay)

# KIV-RI-LA-KIV-RI-LA-YÜ-ZÜ-YOR = FA-FA-SOL-FA-FA-SOL-LA-LA-SOL = F5-F5-G5-F5-F5-G5-A5-A5-G5
z = np.append(yarım_fa_delay,yarım_fa_delay)
z2 = np.append(z,yarım_sol_delay)
z3 = np.append(z2,yarım_fa_delay)
z4 = np.append(z3,yarım_fa_delay)
z5 = np.append(z4,yarım_sol_delay)
z6 = np.append(z5,yarım_la_delay)
z7 = np.append(z6,yarım_la_delay)
z8 = np.append(z7,yarım_sol_delay)

kırmızı_balık = np.append(y6,z8)


sd.play(kırmızı_balık,fs)

wavio.write("kırmızı_balık.wav",kırmızı_balık, fs, sampwidth=2)

In [6]:
# Bu hücre bloğu üst hücrede oluşturulan veya daha önceden oluşturulan seslerin notalarının tespit edilmesi için kullanılır #

## Gerekli kütüphanelerin eklenmesi ##
import numpy as np
import matplotlib.pyplot as plt 
import scipy.signal as sig
import wave
import struct
from scipy.io import wavfile
from scipy.fftpack import fft

## Frekans domaininde incelenmek istenilen sinyalin fft'sinin alınması için gerekli fonksiyonun tanımlanması ##
def frequency_sepectrum(x, sf):
    """
    
     x: zaman domaininde sinyal
     sf: örnekleme frekansı
    
    """
    x = x - np.average(x)  
    n = len(x)
    
    k = np.arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr) 

    frqarr = frqarr[range(n // 2)]  

    x = fft(x) / n 
    x = x[range(n // 2)]

    return frqarr, abs(x)

## İncelemek istediğimiz .wav dosyasının python dosyası içine import edilmesi ##
sr, signal= wavfile.read(r'C:\Users\User\Desktop\B18_ProjeRaporu_v2\B18_Kod\kırmızı_balık.wav') ## !Her bilgisayar için ses dosyasının konumu farklıdır ##

## 44100 hz ile örneklenmiş olan sinyalin daha hızlı incelenebilmesi için M = 2 için downsampling işleminin uygulanması ##
M = 2  
xn_d = signal[np.arange(0, np.size(signal, 0), M)]

## Downsampling işlemi uygulanmış işaretin zaman domaininde genlik ve frekans dizilerinin oluşturulması ##

y = xn_d
t = np.arange(len(y)) / float(sr) 


## İşaretin belirli aralıklarla incelenebilmesi için gerekli koşulların belirlenmesi ##
kosul2 = len(xn_d)
kosul_saniye = int(kosul2 / 11025)
k = int(11025) 

## Notaların frekans değerlerinin tanımlanması ## 
nota_degerler = ([27.500,29.135,30.868,32.703,34.648,36.708,38.891,41.203,43.654,46.249,48.999,51.913,55.000,58.270,
                 61.735,65.406,69.296,73.416,77.782,82.407,87.307,92.499,97.999,103.826,110.000,116.541,123.471,130.813,
                  138.591,146.832,155.563,164.814,174.614,184.997,195.998,207.652,220.000,233.082,246.942,
                 261.626,277.183,293.665,311.127,329.628,349.228,369.994,391.995,415.305,440.000,466.164,493.883,
                 523.251,554.365,587.330,622.254,659.255,698.456,739.989,783.991,830.609,880.000,932.328,987.767,
                 1046.502,1108.731,1174.659,1244.508,1318.510,1396.913,1479.978,1567.982,1661.219,1760.000,1864.655,1975.533
                 ])

## Frekans değerlerine karşılık gelen nota isimlerinin tanımlanması ##
karsilik_harfler = (["A0","A#0","B0","C1","C#1","D1","D#1","E1","F1","F#1","G1","G#1","A1","A#1","B1",
                    "C2","C#2","D2","D#2","E2","F2","F#2","G2","G#2","A2","A#2","B2",
                    "C3","C#3","D3","D#3","E3","F3","F#3","G3","G#3","A3","A#3","B3",
                    "C4","C#4","D4","D#4","E4","F4","F#4","G4","G#4","A4","A#4","B4",
                    "C5","C#5","D5","D#5","E5","F5","F#5","G5","G#5","A5","A#5","B5",
                    "C6","C#6","D6","D#6","E6","F6","F#6","G6","G#6","A6","A#6","B6",])

## İşaretimizin belirli periyotlarda incelenmesi için gerekli döngünün tanımlanması ##
for t in range(1,kosul_saniye+1):
    v = 0
    
    genlik = y[(t-1)*k:t*k] ## İşaretimizin incelenecek aralık için genlik dizisinin belirlenmesi ##
    frekans, genlik2 = frequency_sepectrum(genlik, sr) ## İncelenen aralık için fft alınması ##
    
    kosul_frekans = len(frekans) ## Belirlenen aralıktaki her bir örneğin incelenmesi işlemi için for döngüsü koşulu ##
    
    ## Belirlenen aralıkta fftsi alınmış işaretten baskın frekansın bulunması için tanımlanan for döngüsü ##
    for v in range(kosul_frekans):
        
        tepe = max(genlik2) ## İncelenen dizide maksimum genlik değerinin tespit edilmesi ##
        
        if genlik2[v] == tepe and genlik2[v]>200: ## İncelenen aralıktaki maksimum genliğe karşılık gelen frekans değerinin bulunması ##
            print(t,".nota için frekans değeri\t",frekans[v]/M) ## Bulunan frekans değerinin yazdırılması ##
            flag = True
            i = 0
            while flag: ## Frekans değeri tespit edildikten sonra karşılık gelen nota isminin bulunması için while döngüsü ##
                
                ''' Aşağıdaki if-else bloğunda bulunan frekans değeri 'nota_degerler' dizisindeki değerler ile karşılaştırılarak 
                bulunan frekans değerinin hangi notaya karşılık geldiğini bulmakta ve ekrana yazdırmaktadır.
                '''
                if frekans[v]/M >= nota_degerler[i]: 
                    i = i+1
                    
                else:
                    eq1 = abs(nota_degerler[i]-frekans[v]/M)
                    eq2 = abs(nota_degerler[i-1]-frekans[v]/M)
                    compare = ([eq1,eq2])
                    result = min(compare)
                    
                    if result == eq1:
                        print(t,".nota için harf karşılığı:\t",karsilik_harfler[i]) 
                       
                    else:
                        print(t,".nota için harf karşılığı:\t",karsilik_harfler[i-1])
                       
                    flag = False
        




1 .nota için frekans değeri	 698.0
1 .nota için harf karşılığı:	 F5
2 .nota için frekans değeri	 698.0
2 .nota için harf karşılığı:	 F5
3 .nota için frekans değeri	 784.0
3 .nota için harf karşılığı:	 G5
4 .nota için frekans değeri	 698.0
4 .nota için harf karşılığı:	 F5
5 .nota için frekans değeri	 784.0
5 .nota için harf karşılığı:	 G5
6 .nota için frekans değeri	 880.0
6 .nota için harf karşılığı:	 A5
7 .nota için frekans değeri	 784.0
7 .nota için harf karşılığı:	 G5
8 .nota için frekans değeri	 698.0
8 .nota için harf karşılığı:	 F5
9 .nota için frekans değeri	 698.0
9 .nota için harf karşılığı:	 F5
10 .nota için frekans değeri	 784.0
10 .nota için harf karşılığı:	 G5
11 .nota için frekans değeri	 698.0
11 .nota için harf karşılığı:	 F5
12 .nota için frekans değeri	 698.0
12 .nota için harf karşılığı:	 F5
13 .nota için frekans değeri	 784.0
13 .nota için harf karşılığı:	 G5
14 .nota için frekans değeri	 880.0
14 .nota için harf karşılığı:	 A5
15 .nota için frekans değeri	 880.0
15